# Mod 2 Assessment

### Congratulations on making it to your second assessment! Just a few reminders before you continue:
- This should only take ~70 Minutes, so be sure to manage your time effectively.
- Read the instructions carefully for _specified variable names_.

If there is any confusion on a question, please ask for clarification from an instructor. </br>Though we can't give you the answer, we can help clear up any misunderstandings and get you back on track.

In [4]:
# You'll need these imports to start.
import pymongo
from bson.json_util import loads
from collections import Counter

# put all of your additional imports here:

import pandas as pd
from bs4 import BeautifulSoup as bs
import sqlite3


## Section 0: Coding Readibility (5 Minutes)
Fix the errors in the code below, and rewrite using best practices (hint: don't forget to consider improving variable names).  The final output should BOTH print all variables AND return the list of all the tuples:
```python
def blabla():
for i in range ( 10 ):for j in range( 10 ):for k in range(10): print( i,j,k ) return ( i,j,k )
```

In [6]:
def blabla():
    for i in range(10):
        for j in range(10):
            for k in range(10):
                print(i,j,k)
    return

## Section 1: SQL (10 Minutes)
There is a sqlite3 database in `assets/books.db`. The SQL to create this is also in `assets/books.sql`; if you want to run it manually you can also import this using https://sqliteonline.com/, or run the SQL file directly.  Both have the same schema and data.

The schema has three tables. You can explore the schema in the file posted above.  Please answer the following questions.

Connect to the database using sqlite3. <br/>
Assign `conn` to the sqlite3 connection <br/>
Assign `cur`  to the connection's cursor object

In [6]:
!ls assets

books.db  books.sql  grades.jsonl


In [8]:
conn = sqlite3.connect('./books.db')
cur = conn.cursor()

#### Querying the DB: 

1. How many pages are in the book "Nine Stories"?

In [3]:
# Assign `answer_1` to your final answer
answer_1 = None

2. How many authors are from the USA?

In [1]:
# Assign `answer_2` to your final answer
answer_2 = None

3. How many authors does the book "Professional ASP.NET 4.5 in C# and VB" have?

In [2]:
# Assign `answer_3` to your final answer
answer_3 = None

4. How many pages total have been written by non-American authors?

In [3]:
# Assign `answer_4` to your final answer
answer_4 = None

## Section 2: Object Oriented Programming (15 Minutes)

### Creating a Class
1. Force every new instance of `WeWorkMember` to expect a value which is assigned to the `name` **attribute**.
1. Give every new instance of `WeWorkMember` a `caffeinated` **attribute** that is set to `False`. 
1. Give `WeWorkMember` an **instance method** called `caffeinate()` that prints out "Getting coffee!" and sets the `caffeinated` attribute to `True`.

In [4]:
class WeWorkMember:
    pass

### Inheriting from a Class

1. Have `Staff` and `Student` inherit all methods from `WeWorkMember`
1. Give `Staff` a **static method** called `cheer()` that prints out "Goooooooo Flatiron Students!"
1. Give `Students` a **class method** called `learn()` that takes in an integer and returns that number +1

In [5]:
class Staff():
    pass

class Student():
    pass

## Section 3: APIs & Web Scraping
### APIs (10 Minutes)
Using the API about RuPaul's Drag Race, tell me how many judges of each **`type`** there were in the **first 50 records** the API returns.

API: http://www.nokeynoshade.party/api/judges </br>
Docs: https://drag-race-api.readme.io/docs/get-all-judges

 - Assign `rupaul_resp` to the response of the API request.
 - Ensure the request only returns the first 50 records; the judges will have **`id`** ranging from 1 to 50
 - Do the aggregation in pure Python, Pandas, or SQL -- whatever's easiest for you
 - Assign `judge_count` to a dictionary with the number of judges for each type

In [9]:
rupaul_resp = None

In [6]:
judge_count = None

### Web Scraping (15 Minutes)
Scrape the website [http://books.toscrape.com/](http://books.toscrape.com/) and save it to the `book_resp` variable. Then use BeautifulSoup to convert the response to data we can work with.

- Set the response to a variable named `book_resp`
- Set the BeautifulSoup() object to a variable named `book_soup`

In [8]:
book_resp = None
book_soup = None

## Section 4: NoSQL (10 Minutes)

#### Load data from `assets/grades.json` into Mongo 

(this code is written for you)

In [12]:
# you shouldn't need to edit this cell!
db_name = "mod2db"

with open('assets/grades.jsonl') as f:
    # loads() comes from the bson library
    file_data = [loads(line) for line in f.readlines()]

client = pymongo.MongoClient("mongodb://localhost:27017/")
client.drop_database(db_name)
db = client[db_name]
coll = db["testcoll"]

coll.insert_many(file_data)

#### Answer all of the following questions by querying Mongo and manipulating the results in Python

1. How many records are there total?

In [13]:
# Set `nosql_answer1` to your final answer
nosql_answer1 = None

2. How many students have taken the class with `class_id` = **29**?

In [14]:
# Set `nosql_answer2` to your final answer
nosql_answer2 = None

3. </br>For student **12** in class **23**, what grade did they get on their exam?

In [15]:
# Set `nosql_answer3` to your final answer
nosql_answer3 = None

In [16]:
# for when you're done with this portion, this deletes the data we added.
client.drop_database("mod2db")

## Assessment submission (2 Minutes)
Please save your completed file as `mod2_assessment.ipynb` (UNCHANGED) and upload it using [this form](https://docs.google.com/forms/d/e/1FAIpQLSeY0nH0-DF4sQTQ8vBEsjOwLiQaB-JWopmhh7kFef3TOscrWA/viewform?usp=sf_link)